In [0]:
#programa utilizado para o exemplo de grafos em transporte

In [0]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [0]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("transporteGrafos").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [0]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/iris_bezdekIris.csv,iris_bezdekIris.csv,4551
dbfs:/FileStore/tables/regressaoLinear-1.csv,regressaoLinear-1.csv,564
dbfs:/FileStore/tables/regressaoLinear.csv,regressaoLinear.csv,564
dbfs:/FileStore/tables/transport_nodes_7c826.csv,transport_nodes_7c826.csv,465
dbfs:/FileStore/tables/transport_relationships_c2bfc.csv,transport_relationships_c2bfc.csv,550
dbfs:/FileStore/tables/u.data,u.data,1979173


Carregando o arquivo

In [0]:
nosTransporte="/FileStore/tables/transport_nodes_7c826.csv"  #diretório que contém o arquivo a ser utilizado para os nos

In [0]:
relacionamentosTransporte="/FileStore/tables/transport_relationships_c2bfc.csv"  #diretório que contém o arquivo a ser utilizado

In [0]:
#importando os banco de dados
dfNos = spark.read.format('csv').options(header='true',delimiter=',', inferSchema=True).load(nosTransporte)

In [0]:
dfNos.show()

+----------------+---------+---------+----------+
 id| latitude|longitude|population|
+----------------+---------+---------+----------+
 Amsterdam|52.379189| 4.899431| 821752|
 Utrecht|52.092876| 5.10448| 334176|
 Den Haag|52.078663| 4.288788| 514861|
 Immingham| 53.61239| -0.22219| 9642|
 Doncaster| 53.52285| -1.13116| 302400|
Hoek van Holland| 51.9775| 4.13333| 9382|
 Felixstowe| 51.96375| 1.3511| 23689|
 Ipswich| 52.05917| 1.15545| 133384|
 Colchester| 51.88921| 0.90421| 104390|
 London|51.509865|-0.118092| 8787892|
 Rotterdam| 51.9225| 4.47917| 623652|
 Gouda| 52.01667| 4.70833| 70939|
+----------------+---------+---------+----------+

In [0]:
dfRelacionamentos = spark.read.format('csv').options(header='true',delimiter=',', inferSchema=True).load(relacionamentosTransporte)

In [0]:
dfRelacionamentos.show()

+----------------+----------------+------------+----+
 src| dst|relationship|cost|
+----------------+----------------+------------+----+
 Amsterdam| Utrecht| EROAD| 46|
 Amsterdam| Den Haag| EROAD| 59|
 Den Haag| Rotterdam| EROAD| 26|
 Amsterdam| Immingham| EROAD| 369|
 Immingham| Doncaster| EROAD| 74|
 Doncaster| London| EROAD| 277|
Hoek van Holland| Den Haag| EROAD| 27|
 Felixstowe|Hoek van Holland| EROAD| 207|
 Ipswich| Felixstowe| EROAD| 22|
 Colchester| Ipswich| EROAD| 32|
 London| Colchester| EROAD| 106|
 Gouda| Rotterdam| EROAD| 25|
 Gouda| Utrecht| EROAD| 35|
 Den Haag| Gouda| EROAD| 32|
Hoek van Holland| Rotterdam| EROAD| 33|
+----------------+----------------+------------+----+

Definindo e criando o grafo

In [0]:
#importando as funções para utilizar os grafos
from pyspark.sql.types import *
from graphframes import *  #contém os métodos para serem utilizados no processamento através dos grafos
#obs: a biblioteca graphframes deve ser adicionada ao databricks, pois não é nativa. Para isso acesse New->"Library"-> "Source"-> "Maven Coordinate"-> pesquise por "graphframes" e adicione a biblioteca para cada cluster

In [0]:
#definindo o "Esquema" para cada um dos nós
atributosNo = [
StructField("id", StringType(), True),\
StructField("latitude", FloatType(), True),\
StructField("longitude", FloatType(), True),\
StructField("population", IntegerType(), True)\
]

In [0]:
#importa os dados como nós
nos = spark.read.csv(nosTransporte, header=True,schema=StructType(atributosNo))

Definindo os relacionamentos diretos

In [0]:
#importa os dados como relacionamentos
relaci_direto = spark.read.csv(relacionamentosTransporte, header=True)

In [0]:
relaci_direto.show(5)

+---------+---------+------------+----+
 src| dst|relationship|cost|
+---------+---------+------------+----+
Amsterdam| Utrecht| EROAD| 46|
Amsterdam| Den Haag| EROAD| 59|
 Den Haag|Rotterdam| EROAD| 26|
Amsterdam|Immingham| EROAD| 369|
Immingham|Doncaster| EROAD| 74|
+---------+---------+------------+----+
only showing top 5 rows

Definindo os relacionamentos inversos

In [0]:
#define os relacionamentos com a troca das fontes e destinos
relaci_inverso = (relaci_direto.withColumn("newSrc", relaci_direto.dst).withColumn("newDst", relaci_direto.src)\
.drop("dst", "src")\
.withColumnRenamed("newSrc", "src")\
.withColumnRenamed("newDst", "dst")\
.select("src", "dst", "relationship", "cost"))

In [0]:
relaci_inverso.show(5)

+---------+---------+------------+----+
 src| dst|relationship|cost|
+---------+---------+------------+----+
 Utrecht|Amsterdam| EROAD| 46|
 Den Haag|Amsterdam| EROAD| 59|
Rotterdam| Den Haag| EROAD| 26|
Immingham|Amsterdam| EROAD| 369|
Doncaster|Immingham| EROAD| 74|
+---------+---------+------------+----+
only showing top 5 rows

In [0]:
#cria o df com os relacionamentos diretos e inverso (torna o grafo bidirecional)
relacionamentos=relaci_direto.union(relaci_inverso)

In [0]:
relacionamentos.show()

+----------------+----------------+------------+----+
 src| dst|relationship|cost|
+----------------+----------------+------------+----+
 Amsterdam| Utrecht| EROAD| 46|
 Amsterdam| Den Haag| EROAD| 59|
 Den Haag| Rotterdam| EROAD| 26|
 Amsterdam| Immingham| EROAD| 369|
 Immingham| Doncaster| EROAD| 74|
 Doncaster| London| EROAD| 277|
Hoek van Holland| Den Haag| EROAD| 27|
 Felixstowe|Hoek van Holland| EROAD| 207|
 Ipswich| Felixstowe| EROAD| 22|
 Colchester| Ipswich| EROAD| 32|
 London| Colchester| EROAD| 106|
 Gouda| Rotterdam| EROAD| 25|
 Gouda| Utrecht| EROAD| 35|
 Den Haag| Gouda| EROAD| 32|
Hoek van Holland| Rotterdam| EROAD| 33|
 Utrecht| Amsterdam| EROAD| 46|
 Den Haag| Amsterdam| EROAD| 59|
 Rotterdam| Den Haag| EROAD| 26|
 Immingham| Amsterdam| EROAD| 369|
 Doncaster| Immingham| EROAD| 74|
+----------------+----------------+------------+----+
only showing top 20 rows

In [0]:
#define o grafo 
grafo=GraphFrame(nos, relacionamentos)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2464364679658391> in <module> 
 1 #define o grafo 
 ----> 2 grafo = GraphFrame ( nos , relacionamentos ) 

 /local_disk0/spark-a3127ce7-2094-4069-8227-a31e6200d014/userFiles-8a832040-2725-4d35-9c24-5dd7ef7665f0/addedFile6908688509171570910graphframes_0_7_0_spark2_4_s_2_11-4fc8c.jar/graphframes/graphframe.py in __init__ (self, v, e) 
 87 .format(self.DST, ",".join(e.columns)))
 88 
 ---> 89 self . _jvm_graph = self . _jvm_gf_api . createGraph ( v . _jdf , e . _jdf ) 
 90 
 91 @ property

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 108 def deco ( * a , ** kw ) : 
 109 try : 
 --> 110 return f ( * a , ** kw ) 
 111 except py4j . protocol . Py4JJavaError as e : 
 112 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o891.createGraph.
: java.lang.NoSuchMethodError: scala.Predef$.refArrayOps([Ljava/lang/Object;)Lscala/collection/mutable/ArrayOps;
	at org.graphframes.GraphFrame$.apply(GraphFrame.scala:676)
	at org.graphframes.GraphFramePythonAPI.createGraph(GraphFramePythonAPI.scala:10)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

Explorando o nosso grafo

In [0]:
#encontrando quais são as cidades com mais de 100000 habitantes e menos de 300000
popMedia=grafo.vertices\
.filter("population > 100000 and population < 300000")\
.sort("population")\
.show()

In [0]:
#mostrando a quantidade de caminhos diretos (chegando)
display(grafo.inDegrees)

In [0]:
#mostrando a quantidade de caminhos inversos (saindo)
display(grafo.outDegrees)

In [0]:
#qual é o nó mais "importante" (tem mais caminhos que levam até ele)
total_degree = grafo.degrees
in_degree = grafo.inDegrees
out_degree = grafo.outDegrees


In [0]:
total_degree.show()

In [0]:
total_degree.join(in_degree, "id", how="left")\
.join(out_degree, "id", how="left")\
.fillna(0)\
.sort("inDegree", ascending=False)\
.show()

In [0]:
#realizando consultas
motifs = grafo.find("(Amsterdam)-[e]->(Utrecht)")
display(motifs)

In [0]:
#filtrando os resultados da consulta anterior
filtered = motifs.filter("e.cost < 30")
display(filtered)

In [0]:
#encontrando o menor caminho entre a cidade de Den Haag e alguma das cidades com tamanho médio
origem = "id='Den Haag'"
destino = "population > 100000 and population < 300000 and id <> 'Den Haag'"
resultado = grafo.bfs(origem, destino) #bfs encontra o menor caminho entre dois nós

In [0]:
print(resultado.columns)  #colunas com 'e' representam as arestas (edges) e colunas com 'v' representam os nós (vértices)

In [0]:
#selecionamdo apenas os nós (selecionando quem não começa com e)
colunas = [coluna for coluna in resultado.columns if not coluna.startswith("e")]
resultado.select(colunas).show(5,False)

In [0]:
grafo.vertices\
.filter("population > 100000 and population < 300000")\
.sort("population")\
.show()